In [1]:
#%tensorflow_version 1.15.0
import numpy as np
import imageio
import matplotlib.pyplot as plt
import os
import sys
import tensorflow as tf
import keras
import random
import h5py
#from keras import models
from keras import datasets, layers, models

import cv2 as cv
from PIL import Image

from pathlib import Path
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
tf.__version__

'1.15.0'

In [3]:
from keras.models import load_model

In [4]:
from keras import backend as K

# Metric
def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  
    return iou

In [5]:
model = load_model('../models/my_model_march_28.h5', custom_objects={'iou_coef': iou_coef})













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [6]:
def make_prediction(source_img_path, prediction_path, suture_color=(0, 0, 255)):
    img = cv.imread(source_img_path)
    sagittal_width = int(0.28 * img.shape[1])
    
    img_sagittal = img[:, :sagittal_width]
    pred_list = []
    k = 0
    while k * sagittal_width + sagittal_width < img_sagittal.shape[0]:
        source = img_sagittal[k*sagittal_width: (k+1)*sagittal_width, :sagittal_width]
        square = cv.resize(source, (256, 256))
        square = cv.cvtColor(square, cv.COLOR_BGR2RGB)

        pred_list.append(square)
        k += 1

    source = img_sagittal[-sagittal_width:, :sagittal_width]
    square = cv.resize(source, (256, 256))
    square = cv.cvtColor(square, cv.COLOR_BGR2RGB)

    pred_list.append(square)

    pred_list_array = np.array(pred_list)
    prediction = (model.predict(pred_list_array) > 0.5).astype(np.uint8)
    prediction_big = [cv.resize(x, (sagittal_width, sagittal_width)) for x in prediction]
    
    # Construct an output image
    last_box = prediction_big[-1]
    sag_suture = np.concatenate(prediction_big[0:-1], axis=0)
    sag_show = np.zeros((img_sagittal.shape[0], img_sagittal.shape[1]))
    sag_show[:sag_suture.shape[0]] = sag_suture
    sag_show[-sagittal_width:] = last_box
    
    # Expand the suture line
    sag_show = cv.dilate(sag_show, np.ones((5, 5)))
    
    sag_show_blur = sag_show.astype(np.uint8)
    _, sag_show_blur = cv.threshold(cv.medianBlur(sag_show.astype(np.uint8), 5), 0, 1, cv.THRESH_BINARY)
    sag_show_blur_inv = 1 - sag_show_blur

    img_out = img.copy().astype(np.uint8)
    roi = img_out[:, :sagittal_width]

    green_mask = cv.cvtColor(sag_show_blur_inv, cv.COLOR_GRAY2BGR)
    green_mask[sag_show_blur_inv == 0] = suture_color

    img1_bg = cv.bitwise_and(green_mask, green_mask, mask = sag_show_blur)
    img2_fg = cv.bitwise_and(roi, roi, mask = sag_show_blur_inv)

    img_out_yo = img.copy()
    img_out_yo[:, :sagittal_width] = cv.add(img1_bg, img2_fg)

    cv.imwrite(prediction_path, img_out_yo)
    # print('ok')


In [7]:
make_prediction('../Arbeitung.tif', './ArbeitungLol.tif')

In [8]:
# Batch prediction
input_paths = Path('../batch').glob('*.tif')
output_path = '../batch_output'

for path in tqdm(list(input_paths)):
    output = os.path.join(output_path, path.name)
    make_prediction(str(path), output)

100%|██████████| 35/35 [00:30<00:00,  1.17it/s]
